In [ ]:
import pandas as pd
from scripts.utils import read_gold_data
import scripts.prompts as pr
import scripts.rerank_scripts as rr
from llmusage.hugchat_api_usage import HuggingChat

In [2]:
data_path_name = "data-release" # set data path name

data = read_gold_data(data_path_name)
corpus = data["corpus"]
baseline_queries_train = data["baseline"]["train"]
baseline_queries_dev = data["baseline"]["dev"]
perspective_queries_train = data["perspective"]["train"]
perspective_queries_dev = data["perspective"]["dev"]

# baseline_queries_test = data["baseline"]["test"]
# perspective_queries_test = data["perspective"]["test"]

In [3]:
predictions_path = ""
predictions = pd.read_json(predictions_path, lines=True)

In [4]:
HUGGINGCHAT_USERNAME = "" # insert valid huggingchat username (I believe the email-adress works as well)
HUGGINGCHAT_PASSWORD = "" # insert the corresponding huggingchat password
answer_with = 'dict'

In [5]:
if answer_with == 'list':
    mixtral_huggingchat = HuggingChat(HUGGINGCHAT_USERNAME, HUGGINGCHAT_PASSWORD, system_prompt="Answer with a python list containing all ranked argument ids")
if answer_with == 'dict':
    mixtral_huggingchat = HuggingChat(HUGGINGCHAT_USERNAME, HUGGINGCHAT_PASSWORD, system_prompt="Answer with a python dictionary containing a score between 0 and 1 for each argument id")

created login


In [6]:
queries = perspective_queries_dev # set correct set of queries

In [ ]:
json_path = "" # this path should lead to an empty json file, where the llm demographic scores per query can be saved right after generating them, in case something crashes and local variables are lost
rerank_scores = rr.rerank_scores(predictions, corpus, queries, mixtral_huggingchat, 50, pr.dict_scores_prompt, json_path=json_path)

In [12]:
# this would be used for methods like rr.simple_rerank that do actual reranking and don't return scores
# rerank_scores.to_json("predictions/implicit/list_rerank_sbert_topic_filtered_dev_predictions.jsonl", orient="records", lines=True)

In [ ]:
scores = []
for i in range(len(rerank_scores)):
    row = list(rerank_scores[i])
    scores.append({
        'query_id': queries["query_id"].values[i],
        'rerank_scores': row
    })

In [11]:
rerank_scores_df = pd.DataFrame(scores)

In [12]:
llm_scores_path = ""
rerank_scores_df.to_json(llm_scores_path, orient="records", lines=True)